# Setup

In [1]:
from pandas import *
from pathlib import Path
import pickle
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt

In [2]:
DATA_DIR = "/Users/bradleybrown/Desktop/Waterloo/Courses/3A/CO255/DeepSimplexPivotFinder/data_generation/data/4_euclid"
SAVE_DIR = "spinningup/data/euc_4/euc_4_s0"

In [3]:
data_dir = Path(DATA_DIR)
data_files = list(data_dir.glob("*"))

In [4]:
fname = data_files[0]

In [5]:
with open(fname, "rb") as file:
    data = pickle.load(file)

Aarr = np.array(data['A'])
barr = np.array(data['b'])
carr = np.array(data['c'])

/Users/bradleybrown/opt/anaconda3/envs/co255/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  after removing the cwd from sys.path.
/Users/bradleybrown/opt/anaconda3/envs/co255/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """
/Users/bradleybrown/opt/anaconda3/envs/co255/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


In [6]:
from spinup.utils.test_policy import *

In [7]:
env,get_action = load_policy_and_env(SAVE_DIR, DATA_DIR)



Loading from spinningup/data/euc_4/euc_4_s0/pyt_save/model.pt.




In [8]:
fname = osp.join(SAVE_DIR, 'pyt_save', 'model.pt')
print('\n\nLoading from %s.\n\n'%fname)

model = torch.load(fname)



Loading from spinningup/data/euc_4/euc_4_s0/pyt_save/model.pt.




In [9]:
def p(T, idxs):
    if type(T) == torch.Tensor: T = np.array(T)
    if len(T.shape) == 1: T = T.reshape(23,38)
    print (DataFrame( [[round(r,2) for j,r in enumerate(row) if j in idxs] for i,row in enumerate(T)] ))

In [10]:
def oh(idx, size=37):
    return torch.eye(size)[idx]

In [11]:
def norm(x):
    x = np.array(x)
    return np.sqrt(np.sum(x * x))

In [12]:
model.pi.logits_net.train()

Sequential(
  (0): Linear(in_features=874, out_features=64, bias=True)
  (1): Tanh()
  (2): Linear(in_features=64, out_features=64, bias=True)
  (3): Tanh()
  (4): Linear(in_features=64, out_features=37, bias=True)
  (5): Identity()
)

# Testing

In [13]:
NUM_ITERS = 10000
goal = 26
o, r, d, ep_ret, ep_len, n = env.reset(), 0, False, 0, 0, 0

In [14]:
tab = torch.tensor(o[0], dtype=torch.float32)
cands = torch.tensor(o[1], dtype=torch.bool)
y = oh(goal)
fc = torch.where(cands == False)

In [15]:
tab.requires_grad = True

In [16]:
soft = nn.Softmax()
loss = nn.BCELoss()

In [17]:
before = tab.clone()
optimizer = optim.Adam([tab], lr=1e-2)

In [18]:
for i in range(NUM_ITERS):
    action = soft(model.pi.logits_net(tab))
    
    l = loss(action,y)
    
    l.backward()
    
    optimizer.step()
    optimizer.zero_grad()

/Users/bradleybrown/opt/anaconda3/envs/co255/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [19]:
tab = tab.reshape(23,38)
before = before.reshape(23,38)

In [20]:
delta = tab.detach()-before.detach()

In [25]:
f_obj_d = []
nf_obj_d = []
f_norm_d = []
nf_norm_d = []
targ_norm_d = []
targ_obj_d = []

In [26]:
for i in range(37):
    n_diff = norm(before[:,i].detach())-norm(tab[:,i].detach())
    v_diff = delta[-1,i]
    
    if i == goal:
        targ_obj_d.append(v_diff)
        targ_norm_d.append(n_diff)
        
    elif i in fc[0]:
        f_obj_d.append(v_diff)
        f_norm_d.append(n_diff)
        
    else:
        nf_obj_d.append(v_diff)
        nf_norm_d.append(n_diff)
        

/Users/bradleybrown/opt/anaconda3/envs/co255/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  
/Users/bradleybrown/opt/anaconda3/envs/co255/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
f_obj_d

[tensor(-1.9727),
 tensor(3.3712),
 tensor(2.9147),
 tensor(-2.4269),
 tensor(-1.6722),
 tensor(-3.2956)]

In [28]:
targ_obj_d

[tensor(-1.3147)]

In [29]:
targ_norm_d

[-7.3935833]

In [30]:
f_norm_d

[-11.382032, -11.138738, -11.444481, -10.754679, -8.709473, -10.296619]

In [31]:
nf_norm_d

[-11.981341,
 -14.967484,
 -12.211944,
 -12.226942,
 -9.059896,
 -12.861879,
 -13.968978,
 -10.767195,
 -13.04738,
 -9.050013,
 -11.158063,
 -14.72301,
 -12.614536,
 -11.147387,
 -12.134049,
 -12.414857,
 -13.646106,
 -13.758061,
 -11.7481,
 -11.912448,
 -14.172825,
 -11.844824,
 -11.151147,
 -6.118052,
 -15.12397,
 -10.5315075,
 -12.181516,
 -8.3633,
 -11.597947,
 -10.030359]

In [32]:
nf_obj_d

[tensor(2.4239),
 tensor(-2.6187),
 tensor(-1.1509),
 tensor(-2.7552),
 tensor(-1.7541),
 tensor(1.5145),
 tensor(3.0018),
 tensor(2.1947),
 tensor(-2.6577),
 tensor(-2.0475),
 tensor(2.4693),
 tensor(1.8377),
 tensor(1.9729),
 tensor(-3.3459),
 tensor(0.8990),
 tensor(4.6823),
 tensor(2.7269),
 tensor(-3.1088),
 tensor(-2.3778),
 tensor(1.8995),
 tensor(-2.4208),
 tensor(-3.1608),
 tensor(0.2393),
 tensor(-2.4461),
 tensor(-4.8709),
 tensor(6.5009),
 tensor(-1.8936),
 tensor(3.2136),
 tensor(-3.3779),
 tensor(5.0910)]

In [ ]:
d